In [ ]:
# Install required packages
!pip install ta mplfinance yfinance joblib

In [ ]:
# 📥 Ensure model file exists (download if missing)
import os, requests

model_path = "fibonacci_xgb_model.pkl"
if not os.path.exists(model_path):
    print(f"Model not found locally. Downloading to {model_path}...")
    url = "https://raw.githubusercontent.com/bakshad/Fibonacci-Reversal-bot/main/fibonacci_xgb_model.pkl"
    r = requests.get(url)
    r.raise_for_status()
    with open(model_path, "wb") as f:
        f.write(r.content)
    print("Download complete.")
else:
    print("Model file exists locally.")


In [ ]:
# 📚 Imports (if not already present)
import pandas as pd
import yfinance as yf
import numpy as np
import joblib
import requests
import os
from datetime import datetime
from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator
from ta.volatility import AverageTrueRange
import mplfinance as mpf

# 🔐 Load Telegram & Model
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID  = os.getenv("TELEGRAM_CHAT_ID")
model = joblib.load("fibonacci_xgb_model.pkl")

# 🗃️ Load the full F&O stock list
df_symbols = pd.read_csv("futures_list.csv")
futures_list = df_symbols["Symbol"].dropna().tolist()
print(f"Loaded {len(futures_list)} F&O symbols.")

# 🔔 Telegram helper
def send_telegram(msg, chart_path=None):
    url = (
        f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto"
        if chart_path
        else f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    )
    data = (
        {'chat_id': TELEGRAM_CHAT_ID, 'caption': msg}
        if chart_path
        else {'chat_id': TELEGRAM_CHAT_ID, 'text': msg}
    )
    files = ({'photo': open(chart_path, 'rb')} if chart_path else None)
    resp = requests.post(url, data=data, files=files)
    print("Telegram response:", resp.status_code)

# 🔁 Main loop over every symbol
today = datetime.today().strftime("%Y-%m-%d")
if not os.path.exists("sent_signals.csv"):
    pd.DataFrame(columns=["date","symbol"]).to_csv("sent_signals.csv", index=False)
sent = pd.read_csv("sent_signals.csv")

results = []
for symbol in futures_list:
    try:
        df = yf.download(f"{symbol}.NS", period="6mo", interval="1d", progress=False)
        if df.empty:
            continue

        # 1) Indicators
        df["RSI"] = RSIIndicator(df["Close"]).rsi()
        macd = MACD(df["Close"])
        df["MACD_Hist"] = macd.macd_diff()
        df["ADX"] = ADXIndicator(df["High"], df["Low"], df["Close"]).adx()

        # 2) Fibonacci 0.618 retracement
        high = df["High"].rolling(10).max()
        low = df["Low"].rolling(10).min()
        retracement = high - (high - low) * 0.618
        df["near_618"] = df["Low"] <= retracement

        # 3) ATR-based SL/TP
        atr = AverageTrueRange(df["High"], df["Low"], df["Close"], window=14).average_true_range()
        atr_value = atr.iloc[-1]
        sl_price = float(df["Close"].iloc[-1] - atr_value)
        tp_price = float(df["Close"].iloc[-1] + 3 * atr_value)

        # 4) LSTM trend exhaustion stub
        lstm_ok = True
        if not lstm_ok:
            continue

        latest = df.iloc[-1]
        if latest["near_618"] and latest["RSI"] > 50 and latest["MACD_Hist"] > 0:
            feats = latest[["RSI","MACD_Hist","ADX","Volume"]].fillna(0).values.reshape(1,-1)
            confidence = model.predict_proba(feats)[0][1]

            if confidence >= 0.70 and not ((sent["symbol"]==symbol)&(sent["date"]==today)).any():
                chart_df = df[-50:].copy()
                fib_level = float(retracement.iloc[-1])
                apds = [mpf.make_addplot([fib_level]*len(chart_df), type="line", linestyle="dashed")]
                mpf.plot(
                    chart_df,
                    type="candle",
                    volume=True,
                    addplot=apds,
                    style="default",
                    savefig="chart.png"
                )

                msg = (
                    f"🚨 {symbol} Reversal Alert\n"
                    f"Date: {today}\n"
                    f"Conf: {confidence:.1%}\n"
                    f"RSI: {latest['RSI']:.1f}, MACD: {latest['MACD_Hist']:.2f}\n"
                    f"SL: {sl_price:.2f}, TP: {tp_price:.2f}"
                )
                send_telegram(msg, chart_path="chart.png")

                sent = sent.append({"date":today,"symbol":symbol}, ignore_index=True)
                results.append({"Symbol":symbol,"Conf":confidence})
                sent.to_csv("sent_signals.csv", index=False)

    except Exception as e:
        print(f"Error {symbol}: {e}")

# 📊 Summary
df_summary = pd.DataFrame(results)
print(df_summary)
